### 구글드라이브 연계

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


학습데이터 폴더경로를 설정하세요

In [ ]:
dataPath = "/content/gdrive/MyDrive/LLMStudy/dataset/"

In [ ]:
import torch

In [ ]:
print(torch.__version__)

2.4.1+cu121


### 라이브러리 설치

In [ ]:
# datasets: hugging-face의 인공지능 커뮤니티의 데이터를 불러오고 관리하는 라이브러리
# jsonlines: json 포맷의 데이터를 핸들링하는 라이브러리
!pip install datasets==2.19.0 jsonlines==4.0.0

### huggign face 로그인

In [ ]:
# hugging face 로그인
import huggingface_hub
huggingface_hub.login()

### jsonlines 등 import

In [ ]:
# 행과열을 핸들링하는 라이브러리
import pandas as pd
# json 포맷 데이터를 핸들링하는 라이브러리
import json
import jsonlines
# hugging face 데이터 관리 라이브러리
from datasets import Dataset

1. 학습 데이터 불러오기 및 변환

학습데이터 파일명 설정


In [ ]:
# 데이터 경로 설정 및 불러오기
datasetName = "indata_kor.csv"
jsonFileName = "indata_kor.jsonl"
#datasetName = "indata_kor_add.csv"
#jsonFileName = "indata_kor_add.jsonl"

In [ ]:
print(dataPath)

/content/gdrive/MyDrive/LLMStudy/dataset/


In [ ]:
def csv_to_json(csv_file_path, json_file_path):
    # CSV 파일을 DataFrame으로 읽기★ (한글인 경우 encoding="ms949") 영문인경우 해당 내용 삭제 ★
    #df = pd.read_csv(csv_file_path, encoding="ms949")
    df = pd.read_csv(csv_file_path, encoding="UTF8")
    #df = pd.read_csv(csv_file_path)

    # JSON 파일로 저장
    with open(json_file_path, 'w', encoding='utf-8') as json_file:
        # 각 행을 JSON으로 변환하여 바로 파일에 쓰기
        for index, row in df.iterrows():
            data = {'inputs': row['inputs'], 'response': row['response']}
            json.dump(data, json_file, ensure_ascii=False)
            json_file.write('\n')  # 각 행마다 줄바꿈

# CSV 파일 경로와 JSON 파일 경로 설정
csv_file_path = dataPath + datasetName
json_file_path = dataPath + jsonFileName

# 함수 호출
csv_to_json(csv_file_path, json_file_path)

2. 파인튜닝용 포맷으로 변환

In [ ]:
indataset = []
with jsonlines.open(json_file_path) as f:
    for line in f.iter():
      indataset.append(f'<s>[INST] {line["inputs"]} [/INST] {line["response"]} </s>')
    #   indataset.append(f'<s>### Instruction: \n{line["inputs"]} \n\n### Response: \n{line["response"]}</s>')

# 데이터셋 확인
print('데이터셋 확인')
print(indataset[:5])

# 데이터셋 생성 및 저장
indataset = Dataset.from_dict({"text": indataset})
indataset.save_to_disk(dataPath)

# 데이터셋 info 확인
print('데이터셋 info 확인')
print(indataset)

데이터셋 확인
["<s>[INST] 방탄소년단의 영문명은 무엇인가요? [/INST] BTS는 '방탄소년단( Bang Tan Sonyeondan)'의 영문 약자입니다. 외국인들에게 더 익숙한 표현으로 만들기 위해 BTS의 소속사인 하이브는 BTS를 '비욘드 더 씬( Beyond The Scene)'의 약자라고 설명하고 있습니다. </s>", '<s>[INST] 방탄소년단은 누구인가요? [/INST] 2013년 6월 13일에 데뷔한 빅히트 뮤직 소속 대한민국 7인조 보이 그룹이다이며, 팬 클럽 이름은 아미입니다. </s>', '<s>[INST] BTS의 멤버의 근황은? [/INST] 현재 멤버 슈가, RM, 제이홉, 지민, 뷔, 정국은 병역을 이행 중에 있습니다. </s>', '<s>[INST] 방탄소년단 이름의 의미는 무엇인가요? [/INST] 방탄복이 총알을 막아내는 것처럼, 살아가는 동안 힘든 일을 겪는 10대, 20대가 겪는 힘든 일과 편견을 막아내고 자신들의 음악적 가치를 당당히 지켜내겠다는 의미를 담고 있습니다.  </s>', "<s>[INST] 방탄소년단을 지칭하는 'BTS'는 무엇을 의미합니까? [/INST] 본래 이름인 'BangTan Sonyeundan' 혹은 'Bulletproof Boys'의 준말입니다. 2017년 빅히트 뮤직은 방탄소년단의 공식 로고를 교체하면서 과거와 미래를 아우르는 개념으로 의미를 확장시키고, 'Beyond The Scene'의 준말로 의미를 추가했습니다. </s>"]


Saving the dataset (0/1 shards):   0%|          | 0/11 [00:00<?, ? examples/s]

데이터셋 info 확인
Dataset({
    features: ['text'],
    num_rows: 11
})


In [ ]:
indataset

Dataset({
    features: ['text'],
    num_rows: 11
})

3. huggingface 에 데이터 업로드합니다.

In [ ]:
indataset.push_to_hub("rssaem/rdataset_ko1")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/rssaem/rdataset_ko1/commit/f98566d5aadffcbc9426e46b8d1f8ece78316f07', commit_message='Upload dataset', commit_description='', oid='f98566d5aadffcbc9426e46b8d1f8ece78316f07', pr_url=None, pr_revision=None, pr_num=None)